In [ ]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
posters = pd.read_csv('movie_posters.csv',header=None,names=['movieId','poster_url'],index_col=0)
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
ratings.head()

In [ ]:
user_ratings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
user_ratings.head()

In [ ]:
## Let's Remove Movies which have less than 7 users who rated it. and fill remaining NaN with 0

user_ratings = user_ratings.dropna(thresh=7,axis=1).fillna(0)
user_ratings

In [ ]:
def standardize(row):
    new_row=(row-row.mean())/(row.max()-row.min())
    return new_row
ratings_standardize=user_ratings.apply(standardize)
ratings_standardize

In [ ]:
## Let's Build our Similarity Matrix
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head(50) 

In [ ]:
def get_similar(movie_name,rating):
    similar_ratings = item_similarity_df[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [ ]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

In [ ]:
similar_movies.sum().sort_values(ascending=False).head(20)

In [ ]:
user = [("Grown Ups (2010)",5)]

def check_seen(movie,seen_movies):
    for item,rating in seen_movies:
        if item == movie:
            return True
    
    return False
        
similar_movies = pd.DataFrame()
for movie,rating in user:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

#similar_movies.head(10)
sim_movies=similar_movies.sum().sort_values(ascending=False)
for movie,score in sim_movies.iteritems():
    if not check_seen(movie,user):
        print(movie)
#similar_movies.sum().sort_values(ascending=False).head(20)